# YOLOv11 Custom Object Detection Training  
## Cocoa, Tomato, and Cassava Crop Detection

This notebook presents the end-to-end workflow for training a **custom object detection model using YOLOv11s** for agricultural crop recognition. The primary goal of this project is to accurately detect and localize key crop types—**Cocoa, Tomato, and Cassava**—from real-world farm imagery, enabling applications in precision agriculture and automated crop monitoring.

---

## 📌 Project Overview

The dataset used in this study was **manually annotated using Roboflow**, ensuring high-quality and consistent bounding box labels. A total of **6 object classes** were defined across the three crop categories to capture important visual variations such as plant structure, growth stages, and environmental conditions.

---

## 🧠 Model & Data Details

- **Model Architecture:** YOLOv11s  
- **Annotation Tool:** Roboflow (manual bounding box annotation)  
- **Number of Classes:** 6  
- **Crop Categories:** Cocoa, Tomato, Cassava  
- **Training Objective:** High-accuracy, real-time object detection  

---

## 📂 Notebook Scope

This notebook covers the following stages of the model development lifecycle:

- Dataset preparation and configuration  
- Model training and hyperparameter setup  
- Performance evaluation using Precision, Recall, and mAP metrics  
- Inference setup for deployment readiness  

---

## 🚀 Deployment Readiness

The resulting trained model is designed to be **scalable and deployment-ready**, making it suitable for integration into:

- Edge devices and embedded systems  
- Mobile and web-based applications  
- Intelligent agricultural monitoring platforms 


In [1]:
import torch
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

PyTorch version: 2.4.1
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [2]:
from ultralytics import YOLO

model = YOLO("yolo11s.pt")

model.train(
    # Dataset
    data="/Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/data.yaml",

    # Core training
    epochs=100,              # small dataset → needs time
    imgsz=640,
    batch=16,
    device="mps",

    # Optimizer (DO NOT use auto here)
    optimizer="SGD",         # better for small custom datasets
    lr0=0.01,
    momentum=0.937,
    weight_decay=0.0005,

    # Disable early stopping (CRITICAL)
    patience=0,

    # Encourage recall during learning
    iou=0.5,
    max_det=200,

    # Controlled augmentation (dataset already augmented)
    mosaic=0.3,
    mixup=0.0,
    copy_paste=0.0,
    erasing=0.0,

    # Stability
    warmup_epochs=3,
    close_mosaic=10,

    # Logging / outputs
    plots=True,
    verbose=True,
    save=True,
)

Ultralytics 8.3.237 🚀 Python-3.8.20 torch-2.4.1 MPS (Apple M1 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=200, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=0.3, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=0, perspectiv

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x17fdb6760>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [5]:
from ultralytics import YOLO

# Load the best model
model = YOLO("runs/detect/train/weights/best.pt")

# Path to a test image (NO extra quotes)
image_path = '/Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/test/images'

# Run inference
results = model(
    image_path,
    conf=0.2,        
    iou=0.5,
    save=True
)


image 1/30 /Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/test/images/CBB-10-_JPG.rf.5f969fd3ae48580913ba44a52ff192a7.jpg: 640x640 1 Cassava Bacteria Blight, 122.2ms
image 2/30 /Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/test/images/CBB-20-_JPG.rf.2f4e3254579e7dce1f962b0e4adfc8ca.jpg: 640x640 1 Cassava Bacteria Blight, 75.2ms
image 3/30 /Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/test/images/CBB-3-_JPG.rf.25cb4967af7ed121ac297ddd157809e2.jpg: 640x640 1 Cassava Bacteria Blight, 75.7ms
image 4/30 /Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/test/images/CBB-65-_JPG.rf.e529d03cfab788ae66d04bfa79f90c74.jpg: 640x640 1 Cassava Bacteria Blight, 80.3ms
image 5/30 /Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/test/images/CBB-68-_JPG.rf.298f1af1c225ea488d0a606b16a26bad.jpg: 640x640 1 Cassava Bacteria Blight, 73.8ms
image 6/30 /Users/fola-ai/Projects/computer vision/farmeyes_yolov11/data/test/images/CBB-69-_JPG.rf

### Final verdict

✅ Final Results & Evaluation

The YOLOv11s model was trained to detect six crop disease classes across Tomato, Cocoa, and Cassava using bounding-box annotations.
Evaluation was performed on a held-out validation set to measure generalization.

🔍 Validation Performance

Precision: 0.77

Recall: 0.73

mAP@0.50: 0.74

mAP@0.50–0.95: 0.45

These results indicate that the model reliably detects most disease instances while maintaining good prediction accuracy.

🧪 Test Inference

The best-performing model (best.pt) was tested on unseen images using:

conf=0.2, iou=0.5


Predictions show accurate localization and correct class identification for most cases.

📌 Conclusion

Given the dataset size and class complexity, the model achieves strong and practical performance.
Further improvements are expected primarily from adding more data, especially for underrepresented classes.